In [1]:
%pylab inline
from fastai.vision import *
from pathlib import Path
import pandas as pd
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
path=Path('data')

In [3]:
path.ls()

[WindowsPath('data/Anno'),
 WindowsPath('data/Eval'),
 WindowsPath('data/img'),
 WindowsPath('data/input_img.jpg'),
 WindowsPath('data/models'),
 WindowsPath('data/README.txt')]

In [4]:
f=open(path/"Anno/list_category_cloth.txt", "r")
contents =f.read().splitlines()

In [5]:
num_classes = int(contents[0])
print(num_classes)

50


In [6]:
classes = []
for i in range(2,len(contents)):
    classes.append(contents[i].split(" ")[0])
print(classes)

['Anorak', 'Blazer', 'Blouse', 'Bomber', 'Button-Down', 'Cardigan', 'Flannel', 'Halter', 'Henley', 'Hoodie', 'Jacket', 'Jersey', 'Parka', 'Peacoat', 'Poncho', 'Sweater', 'Tank', 'Tee', 'Top', 'Turtleneck', 'Capris', 'Chinos', 'Culottes', 'Cutoffs', 'Gauchos', 'Jeans', 'Jeggings', 'Jodhpurs', 'Joggers', 'Leggings', 'Sarong', 'Shorts', 'Skirt', 'Sweatpants', 'Sweatshorts', 'Trunks', 'Caftan', 'Cape', 'Coat', 'Coverup', 'Dress', 'Jumpsuit', 'Kaftan', 'Kimono', 'Nightdress', 'Onesie', 'Robe', 'Romper', 'Shirtdress', 'Sundress']


In [7]:
f=open(path/"Anno/list_category_img.txt", "r")
labels_content =f.read().splitlines()

f=open(path/"Eval/list_eval_partition.txt", "r")
val_split =f.read().splitlines()

In [8]:
labels = []
for i in range(2,len(labels_content)):
    labels.append(np.append(labels_content[i].split(),val_split[i].split()[1]))  

In [9]:
labels_df = pd.DataFrame(labels)
labels_df['is_valid'] =  labels_df.apply(lambda row: False if row[2] == "train" else True, axis=1)

In [10]:
labels_df

,0,1,2,is_valid
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,3,train,False
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,3,train,False
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,3,val,True
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,3,train,False
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,3,test,True
...,...,...,...,...
289217,img/Paisley_Print_Babydoll_Dress/img_00000050.jpg,41,train,False
289218,img/Paisley_Print_Babydoll_Dress/img_00000051.jpg,41,train,False
289219,img/Paisley_Print_Babydoll_Dress/img_00000052.jpg,41,train,False
289220,img/Paisley_Print_Babydoll_Dress/img_00000053.jpg,41,train,False


In [11]:
labels_df.rename(columns = {0:'filename',1:'label', 2:'val_label'}, inplace = True) 

In [12]:
#trained_labels = labels_df.groupby('label').apply(lambda x: x.sample(n=1000, replace=True))

In [13]:
#trained_labels.head()

In [14]:
def get_data(bs=8, size=224):
    src = ImageList.from_df(labels_df, path)
    src = src.split_from_df(col='is_valid')
    #src = src.split_by_rand_pct(0.2)
    src = src.label_from_df(cols='label')
    src = src.transform(get_transforms(), size=size)
    return src.databunch(path=path, bs=bs).normalize(imagenet_stats)

In [15]:
data = get_data()

In [16]:
#data=ImageDataBunch.from_df(path=path/"", df=labels_df, fn_col='filename', label_col='label', 
    #                       ds_tfms=get_transforms(), size=256, bs=8).normalize(imagenet_stats)

In [17]:
learn=cnn_learner(data, models.resnet34, metrics=accuracy).to_fp16()

In [18]:
learn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 112, 112]       9,408      False     
______________________________________________________________________
BatchNorm2d          [64, 112, 112]       128        True      
______________________________________________________________________
ReLU                 [64, 112, 112]       0          False     
______________________________________________________________________
MaxPool2d            [64, 56, 56]         0          False     
______________________________________________________________________
Conv2d               [64, 56, 56]         36,864     False     
______________________________________________________________________
BatchNorm2d          [64, 56, 56]         128        True      
______________________________________________________________________
ReLU                 [64, 56, 56]         0          False     
___________________________________________________

In [19]:
learn.freeze()

In [20]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [ ]:
learn.recorder.plot()

In [ ]:
lr=5e-3
learn.fit_one_cycle(8, slice(lr/10, lr))

In [ ]:
interp=ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(k=6, figsize=(12, 8))

In [ ]:
learn.save("trained_model_resnet50_15epoch_50k")

## Showing results

In [ ]:
with torch.no_grad():
    x,y = next(iter(learn.data.valid_dl))
    yhat = learn.model(x.to('cuda'))

In [ ]:
x.shape, y[0].shape, yhat.shape